In [ ]:
import os
import pydicom
import pandas as pd
import numpy as np
import cv2
import math
data_dir = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/"
patients = os.listdir(data_dir)

In [ ]:
def average(l):
    return sum(l)/len(l)
    
def layers(l,n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

m2 = [22, 23, 24, 36, 37, 38, 39, 40, 50, 51, 52, 53, 54, 55, 56, 65, 66, 67, 68, 69, 70, 81, 82, 83, 84, 97, 98]
m3 = [19, 20, 21, 25, 26, 33, 34, 35, 41, 42, 49]
m4 = [18, 27, 28]
m5 = [17]

def adjuster(file):
    if len(file) in m5:
        n = 5
    elif len(file) in m4:
        n = 4
    elif len(file) in m3:
        n = 3
    elif len(file) in m2:
        n = 2
    else:
        n = 1
    new_file = []
    for i in range(len(file)):
        for j in range(n):
            new_file.append(file[i])
    return new_file       

def decider(length, layer_number):
    return math.ceil(length/layer_number)
   
def adjuster2(layers, layer_number):
    if len(layers) == layer_number - 1:
        layers.append(layers[-1])

In [ ]:
layer_number=16
c=1
Input_Values = []
for p in patients[:]:
    if p != '00109' and p != '00123' and p != '00709':
        scan=[]

        flair = [
            cv2.resize(pydicom.read_file(data_dir + p + '/FLAIR/'+ layer).pixel_array,(64,64)) 
            for layer in os.listdir(data_dir + p + '/FLAIR/')]

        new_flair=[]
        flair = adjuster(flair)
        layer_size = decider(len(flair) , layer_number)
        for layer_chunk in layers(flair, layer_size):
            layer_chunk = list(map(average, zip(*layer_chunk)))
            new_flair.append(layer_chunk)
        adjuster2(new_flair, layer_number)

        T1w = [
            cv2.resize(pydicom.read_file(data_dir + p + '/T1w/'+ layer).pixel_array,(64,64)) 
            for layer in os.listdir(data_dir + p + '/T1w/')]

        new_T1w = []
        T1w=adjuster(T1w)
        layer_size = decider(len(T1w) , layer_number)
        for layer_chunk in layers(T1w, layer_size):
            layer_chunk = list(map(average, zip(*layer_chunk)))
            new_T1w.append(layer_chunk)

        adjuster2(new_T1w, layer_number)

        T1wCE = [
            cv2.resize(pydicom.read_file(data_dir + p + '/T1wCE/'+ layer).pixel_array,(64,64)) 
            for layer in os.listdir(data_dir + p + '/T1wCE/')]

        new_T1wCE = []
        T1wCE=adjuster(T1wCE)
        layer_size = decider(len(T1wCE) , layer_number)
        for layer_chunk in layers(T1wCE, layer_size):
            layer_chunk = list(map(average, zip(*layer_chunk)))
            new_T1wCE.append(layer_chunk)

        adjuster2(new_T1wCE, layer_number)

        T2w = [
            cv2.resize(pydicom.read_file(data_dir + p + '/T2w/'+ layer).pixel_array,(64,64)) 
            for layer in os.listdir(data_dir + p + '/T2w/')]

        new_T2w=[]
        T2w=adjuster(T2w)
        layer_size = decider(len(T2w) , layer_number)
        for layer_chunk in layers(T2w, layer_size):
            layer_chunk = list(map(average, zip(*layer_chunk)))
            new_T2w.append(layer_chunk)

        adjuster2(new_T2w, layer_number)

        for i in range(0,layer_number):
            x=[]
            for j in range(64):
                y =[]
                for k in range(64):
                    channel=[]
                    channel.append(new_flair[i][j][k])
                    channel.append(new_T1w[i][j][k])
                    channel.append(new_T1wCE[i][j][k])
                    channel.append(new_T2w[i][j][k])
                    y.append(channel)
                x.append(y)
            scan.append(np.array(x))
        Input_Values.append(scan)
        print(str(c) +' Done')
        c+=1

np.shape(Input_Values)

In [ ]:
df = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')
df = df.drop([71,81,488])
Output = df['MGMT_value']
np.shape(Output)

In [ ]:
Output_Values = []
for i in range(585):
    if i != 71 and i != 81 and i != 488:
        Output_Values.append(Output[i])
np.array(Output_Values)

In [ ]:
Output_Values = np.reshape(Output_Values,(582,1))
Input_Values = np.array(Input_Values)
Output_Values = np.array(Output_Values)
np.save('./Input_Values_64x64',Input_Values)
print(np.shape(Input_Values))
print(np.shape(Output_Values))

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv3D, MaxPooling3D, BatchNormalization, Activation, Dropout, Flatten, Dense
from tensorflow.keras.models import Sequential, load_model
import matplotlib.pyplot as plt

In [ ]:
model = Sequential()
model.add(Conv3D(32, (3, 3, 3), activation = "relu", input_shape=(16, 64, 64, 4)))
model.add(MaxPooling3D(pool_size=2))
model.add(BatchNormalization())
model.add(Conv3D(64, (3, 3, 3), activation = "relu"))
model.add(MaxPooling3D(pool_size=(1, 2, 2)))
model.add(BatchNormalization())
model.add(Conv3D(128, (3, 3, 3), activation = "relu"))
model.add(MaxPooling3D(pool_size=(1, 2, 2)))
model.add(BatchNormalization())
model.add(Conv3D(256, (3, 3, 3), activation = "relu"))
model.add(MaxPooling3D(pool_size=(1, 2, 2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(256, activation = "relu" ))
model.add(Dropout(0.3))
model.add(Dense(1, activation = "sigmoid" ))

model.summary()

In [ ]:
opt = tf.keras.optimizers.SGD(learning_rate=0.0003)
model.compile( loss='binary_crossentropy' , metrics=['accuracy'], optimizer=opt)

In [ ]:
history = model.fit(Input_Values,Output_Values,validation_split=0.1, epochs=500, batch_size=20)

print(history.history.keys())

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Training set', 'Validation set'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Training set', 'Validation set'], loc='upper left')
plt.show()

In [ ]:
model.save('./Model5')